In [22]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

In [23]:
#primer archivo a procesar e insertar en la base de datos
df_20200413=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200413_csv.csv')

In [24]:
#segundo archivo a procesar e insertar en la base de datos
sheet2=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200419_csv.csv')

#tercer archivo a procesar e insertar en la base de datos

In [25]:
sheet1=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200426_csv.csv')

In [31]:
#cuarto archivo a procesar e insertar en la base de datos
df_20200503=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200503_csv.csv')

In [ ]:
semana_20200518=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200518_csv.csv')

In [26]:
sucursal=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\Datasets relevamiento precios\sucursal.csv')

In [ ]:
producto_parquet=pd.read_csv(r'C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\producto_parquet_csv.csv')

In [ ]:
sheet1.info()

In [ ]:
sheet1.isnull().sum()

si es nulo precio se agrega un 1 a la columna col_added

In [27]:
def add_a_b(a, b, c):
            
            if (a==True or b==True or c==True):
                return 1
            else:
                return 0


In [28]:

def columna_boolean_nan(sheet1):
    COL1= sheet1["precio"].isnull()
    sheet1["bool1_precio"] = COL1
    COL2= sheet1["producto_id"].isnull()
    sheet1["bool2_producto_id"] = COL2
    COL3=sheet1["sucursal_id"].isnull()
    sheet1["bool3_sucursal_id"] = COL3
        
    sheet1["col_added"] = sheet1.apply(lambda x: add_a_b(x["bool1_precio"], x["bool2_producto_id"],x["bool3_sucursal_id"]), axis = 1)
    sheet1=sheet1.drop(columns=["bool1_precio","bool2_producto_id","bool3_sucursal_id"])
    return sheet1

In [ ]:
new_df

In [ ]:
sheet1

In [29]:
def cortar_columna_sucursal_id_precio(df_precio):
    df_precio["sucursal_id"]=df_precio["sucursal_id"].astype('str')
    new2 = df_precio["sucursal_id"].str.split(" ", n = 1, expand = True)
    df_precio["sucursal_id"]=new2[0]
    return df_precio

In [8]:
def array_col_id_unicos(df_sucursal,nombre_de_columna):
    list=[]
    df_sucursal[nombre_de_columna]=df_sucursal[nombre_de_columna].astype('str')
    new2 = df_sucursal[nombre_de_columna].str.split("-", n = 2, expand = True)
    list=df_sucursal[nombre_de_columna]
    col_unicos=list.unique()
    return col_unicos

saco el punto decimal que tiene la columna id producto
completa con ceros a la izquierda

In [9]:
def borrar_nan_producto_id(sheet1):
    sheet1["producto_id"]=sheet1["producto_id"].astype('str')
    rep = {'NaN':'0.0','nan':'0.0',}
    sheet1["producto_id"] =sheet1["producto_id"].replace(rep, regex=True)
    new1= sheet1["producto_id"].str.split(".", n = 1, expand = True)
    sheet1["producto_id"]=new1[0]
    sheet1["producto_id"]= sheet1["producto_id"].str.zfill(13)
    return sheet1

Primer archivo

In [11]:

new_df=columna_boolean_nan(df_20200413)
new_df1=borrar_nan_producto_id(new_df)
sheet1=new_df1.copy()

In [ ]:
df_20200413.isnull().sum()

filtro los nan buscando en la columna col_added los 1 (unos)

In [ ]:
sheet1[sheet1["col_added"]==1].shape

In [12]:
sheet1["numero_archivo"]=1

In [ ]:
sheet1

In [30]:
try :
    mi_conexion=sqlite3.connect("miprimera.bd")
    cursor_obj = mi_conexion.cursor()
except Exception as ex:
    print(ex)

In [37]:
sheet2["numero_archivo"]=2

In [39]:
df_precios_=pd.read_sql("SELECT * FROM precios", mi_conexion)
precios_concat = pd.concat([df_precios_, sheet2])

In [46]:
df_precios_

,sucursal_id,producto_id,precio,col_added,numero_archivo
0,2-1-014,0000000001663,29.90,0,1
1,2-1-032,0000000002288,29.90,0,1
2,2-1-096,0000000002288,39.90,0,1
3,9-1-686,0000000205870,499.99,0,1
4,9-2-248,0000000205870,519.99,0,1
...,...,...,...,...,...
472161,25-1-1,9569753142128,139.99,0,1
472162,25-1-1,9795403001143,34.99,0,1
472163,5-1-3,9990385651922,312.50,0,1
472164,5-1-3,9990385651939,312.50,0,1


In [48]:
sheet2.to_sql('precios_temp', con=mi_conexion, index=False, index_label='id', if_exists='replace')

df_precios_=pd.read_sql("SELECT sucursal_id,producto_id,precio,numero_archivo FROM precios UNION ALL SELECT sucursal_id,producto_id,precio,numero_archivo FROM precios_temp", mi_conexion)


In [49]:
df_precios_

,sucursal_id,producto_id,precio,numero_archivo
0,2-1-014,0000000001663,29.90,1
1,2-1-032,0000000002288,29.90,1
2,2-1-096,0000000002288,39.90,1
3,9-1-686,0000000205870,499.99,1
4,9-2-248,0000000205870,519.99,1
...,...,...,...,...
930704,2001-01-25 00:00:00,9569753142128.0,139.99,2
930705,2001-01-25 00:00:00,9795403001143.0,34.99,2
930706,2003-01-05 00:00:00,9990385651922.0,312.50,2
930707,2003-01-05 00:00:00,9990385651939.0,312.50,2


In [47]:
sheet2

,sucursal_id,producto_id,precio,numero_archivo
0,2-1-184,2288.0,29.90,2
1,2-1-206,2288.0,39.90,2
2,9-1-430,205870.0,499.99,2
3,9-2-107,205870.0,539.99,2
4,5218-03-09 00:00:00,205870.0,539.99,2
...,...,...,...,...
458538,2001-01-25 00:00:00,9569753142128.0,139.99,2
458539,2001-01-25 00:00:00,9795403001143.0,34.99,2
458540,2003-01-05 00:00:00,9990385651922.0,312.50,2
458541,2003-01-05 00:00:00,9990385651939.0,312.50,2


In [41]:
precios_concat[precios_concat["numero_archivo"]==1]

,sucursal_id,producto_id,precio,col_added,numero_archivo
0,2-1-014,0000000001663,29.90,0.0,1
1,2-1-032,0000000002288,29.90,0.0,1
2,2-1-096,0000000002288,39.90,0.0,1
3,9-1-686,0000000205870,499.99,0.0,1
4,9-2-248,0000000205870,519.99,0.0,1
...,...,...,...,...,...
472161,25-1-1,9569753142128,139.99,0.0,1
472162,25-1-1,9795403001143,34.99,0.0,1
472163,5-1-3,9990385651922,312.50,0.0,1
472164,5-1-3,9990385651939,312.50,0.0,1


In [15]:
sheet1.to_sql('precios', con=mi_conexion, index=False, index_label='id', if_exists='replace')

472166

In [16]:
df_final=sheet1[sheet1["col_added"]==0]

In [17]:
df_final.to_sql('precios_final', con=mi_conexion, index=False, index_label='id', if_exists='replace')

472151

In [ ]:
sheet1

Segundo archivo

In [18]:
sheet2["col_added"]=0
new_df=columna_boolean_nan(sheet2)
new_df1=borrar_nan_producto_id(new_df)
sheet2=new_df1.copy()
new_df1=cortar_columna_sucursal_id_precio(sheet2)
"""comprobar sucursalId si no coincide con id de la tabla sucursal se marca con 1 la columna col_added"""
list_suc_unicos=[]
list_suc_prec=[]
sheet2=new_df1.copy()
list_suc_unicos=array_col_id_unicos(sucursal,"id")
list_suc_prec=array_col_id_unicos(sheet2,"sucursal_id")

lis_bin=[]
lis_id=[]
for e in list_suc_prec:
    for i in list_suc_unicos:
        if(e == i):
            result_id=0
        else:
            result_id=1
            
        if (result_id == 0):
            break  
    lis_bin.append(result_id)
    lis_id.append(e)
list_zip=list(zip(lis_id,lis_bin))
DF=pd.DataFrame()
for e,num in enumerate(list_zip):
    #print(e,num[0],num[1])
    if(num[1]==1):
        DF=sheet2[sheet2["sucursal_id"]==num[0]]
        DF["col_added"]=1
        sheet2[sheet2["sucursal_id"]==num[0]]=DF

C:\Users\ROXI\AppData\Local\Temp\ipykernel_1924\3496743843.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF["col_added"]=1


In [19]:
sheet2["numero_archivo"]=2

In [52]:
sheet2

,sucursal_id,producto_id,precio,col_added,numero_archivo
0,2-1-184,0000000002288,29.90,0,2
1,2-1-206,0000000002288,39.90,0,2
2,9-1-430,0000000205870,499.99,0,2
3,9-2-107,0000000205870,539.99,0,2
4,5218-03-09,0000000205870,539.99,1,2
...,...,...,...,...,...
458538,2001-01-25,9569753142128,139.99,1,2
458539,2001-01-25,9795403001143,34.99,1,2
458540,2003-01-05,9990385651922,312.50,1,2
458541,2003-01-05,9990385651939,312.50,1,2


In [20]:
sheet2.to_sql('precios_aux', con=mi_conexion, index=False, index_label='id', if_exists='replace')

458543

In [258]:
df_precios_=pd.read_sql("SELECT * FROM precios", mi_conexion)

In [182]:
precios_concat = pd.concat([df_precios_, sheet2])

In [183]:
precios_concat.to_sql('precios', con=mi_conexion, index=False, index_label='id', if_exists='replace')

930709

In [ ]:

#precios_final=pd.read_sql("SELECT * FROM precios_final WHERE numero_archivo=1", mi_conexion)

In [ ]:
cursor_obj.execute("DROP TABLE IF EXISTS precios_tb_final")
table = "CREATE TABLE precios_tb_final  as SELECT precios_aux.sucursal_id,precios_aux.producto_id,precios_aux.precio,precios_aux.col_added	,precios_aux.numero_archivo FROM precios_aux LEFT JOIN(SELECT * FROM precios_final) AS prec USING(sucursal_id,producto_id)" 
cursor_obj.execute(table)

In [ ]:
df2=sheet2[sheet2["col_added"]==0]
df_final.to_sql('precios_final', con=mi_conexion, index=False, index_label='id', if_exists='replace')

Tercer archivo

In [16]:
sheet1["col_added"]=0
new_df=columna_boolean_nan(sheet1)
"""cambia el valor nan por 000000000000 """
new_df1=borrar_nan_producto_id(new_df)
sheet1=new_df1.copy()
"""comprobar sucursalId si no coincide con id de la tabla sucursal"""
list_suc_unicos=[]
list_suc_prec=[]
list_suc_unicos=array_col_id_unicos(sucursal,"id")
list_suc_prec=array_col_id_unicos(sheet1,"sucursal_id")

lis_bin=[]
lis_id=[]
for e in list_suc_prec:
    for i in list_suc_unicos:
        if(e == i):
            result_id=0
        else:
            result_id=1
            
        if (result_id == 0):
            break  
    lis_bin.append(result_id)
    lis_id.append(e)
list_zip=list(zip(lis_id,lis_bin))
DF=pd.DataFrame()
for e,num in enumerate(list_zip):
    #print(e,num[0],num[1])
    if(num[1]==1):
        DF=sheet1[sheet1["sucursal_id"]==num[0]]
        DF["col_added"]=1
        sheet1[sheet1["sucursal_id"]==num[0]]=DF

In [17]:
sheet1["numero_archivo"]=3

In [ ]:
sheet1.to_sql('precios_aux', con=mi_conexion, index=False, index_label='id', if_exists='replace')

df_precios_=pd.read_sql("SELECT * FROM precios", mi_conexion)
precios_concat = pd.concat([df_precios_, sheet1])
precios_concat.to_sql('precios', con=mi_conexion, index=False, index_label='id', if_exists='replace')



In [ ]:
cursor_obj.execute("DROP TABLE IF EXISTS precios_tb_final")
table = "CREATE TABLE precios_tb_final  as SELECT precios_aux.sucursal_id,precios_aux.producto_id,precios_aux.precio,precios_aux.col_added	,precios_aux.numero_archivo FROM precios_aux LEFT JOIN(SELECT * FROM precios_final) AS prec USING(sucursal_id,producto_id)" 
cursor_obj.execute(table)

In [ ]:
df_final=sheet1[sheet1["col_added"]==0]
df_final.to_sql('precios_final', con=mi_conexion, index=False, index_label='id', if_exists='replace')

In [ ]:
new_df=columna_boolean_nan(sheet1)
new_df1=borrar_nan_producto_id(new_df)
sheet1=new_df1.copy()

In [12]:
sheet1["col_added"]=0
new_df=columna_boolean_nan(sheet1)
"""cambia el valor nan por 000000000000 """
new_df1=borrar_nan_producto_id(new_df)
sheet1=new_df1.copy()
"""comprobar sucursalId si no coincide con id de la tabla sucursal"""
list_suc_unicos=[]
list_suc_prec=[]
list_suc_unicos=array_col_id_unicos(sucursal,"id")
list_suc_prec=array_col_id_unicos(sheet1,"sucursal_id")

lis_bin=[]
lis_id=[]
for e in list_suc_prec:
    for i in list_suc_unicos:
        if(e == i):
            result_id=0
        else:
            result_id=1
            
        if (result_id == 0):
            break  
    lis_bin.append(result_id)
    lis_id.append(e)
list_zip=list(zip(lis_id,lis_bin))
DF=pd.DataFrame()
for e,num in enumerate(list_zip):
    #print(e,num[0],num[1])
    if(num[1]==1):
        DF=sheet1[sheet1["sucursal_id"]==num[0]]
        DF["col_added"]=1
        sheet1[sheet1["sucursal_id"]==num[0]]=DF

In [13]:
sheet1.to_sql('precios_aux', con=mi_conexion, index=False, index_label='id', if_exists='replace')

NameError: name 'mi_conexion' is not defined

In [ ]:
list_producto_unicos=[]
list_producto_prec=[]
list_producto_unicos=array_col_id_unicos(df1,"id")
list_producto_prec=array_col_id_unicos(sheet1,"producto_id")
"""comprobar producto_id si no coincide con id de la tabla sucursal se marca con 1 la columna col_added"""
lis_bin=[]
lis_id=[]
for e in list_producto_prec:
    for i in list_producto_unicos:
        if(e == i):
            result_id=0
        else:
            result_id=1
            
        if (result_id == 0):
            break  
    lis_bin.append(result_id)
    lis_id.append(e)
list_zip=list(zip(lis_id,lis_bin))
DF=pd.DataFrame()
for e,num in enumerate(list_zip):
    #print(e,num[0],num[1])
    if(num[1]==1):
        DF=sheet1[sheet1["producto_id"]==num[0]]
        DF["col_added"]=1
        sheet1[sheet1["producto_id"]==num[0]]=DF

In [ ]:
sheet1[sheet1["col_added"]==1]

In [ ]:
sheet2.isnull().sum()

In [ ]:
sheet2[sheet2["col_added"]==1]

In [ ]:
sheet2.info()

In [ ]:
sheet2.isnull().sum()

In [ ]:
sheet2

In [ ]:
ruta_archivo_json = r"C:\Users\ROXI\OneDrive\Escritorio\dataset procesados\jupyter\precios_semana_20200503_csv.csv"
datos_semana_20200503= pd.read_json(ruta_archivo_json,encoding="utf-8")